# Stocks behavior

In [1]:
from pylab import *
import multiprocessing as mp
import pickle
import datetime
import gzip

In [2]:
### List of order types: 
### "B" = 1 -- Add buy order
### "S" = 2 -- Add sell order
### "E" = 3 -- Execute outstanding order in part
### "C" = 4 -- Cancel outstanding order in part
### "F" = 5 -- Execute outstanding order in full
### "D" = 6 -- Delete outstanding order in full
### "X" = 7 -- Bulk volume for the cross event
### "T" = 8 -- Execute non-displayed order

In [3]:
tickers = ["AAL", "AAPL","ADBE","ADI", "ADP", "ADSK","AKAM","ALXN","AMAT","AMGN",
        "AMZN","ATVI","AVGO","BBBY","BIDU","BIIB","BMRN","CA",  "CELG","CERN",
        "CHKP","CHRW","CHTR","CMCSA","COST","CSCO","CTSH","CTXS","DISCA","DISH",
         "DLTR","EA",  "EBAY","EQIX","ESRX","EXPD","FAST","FB",  "FISV","FOXA",
         "GILD","GOOG","GRMN","HSIC","ILMN","INTC","INTU","ISRG","JD",  "KHC",
         "KLAC","LBTYA","LLTC","LMCA","LRCX","LVNTA","MAR","MAT","MDLZ","MNST",
         "MSFT","MU",  "MYL", "NFLX","NTAP","NVDA","NXPI","ORLY","PAYX","PCAR",
         "PCLN","QCOM","REGN","ROST","SBAC","SBUX","SIRI","SNDK","SPLS","SRCL",
         "STX", "SYMC","TRIP","TSCO","TSLA","TXN", "VIAB","VIP", "VOD", "VRSK",
         "VRTX","WDC", "WFM", "WYNN","XLNX","YHOO"]

In [52]:
def plot_stocks(ticker):
    
    fig = figure(figsize=(16,9))
        
    days=['07','08','09','10','11']
    
    for day in days:

# -----------------------------------------------------------------------------------------------------------------------

        ### Load data
        print(ticker, day)

        data = genfromtxt(gzip.open('../ITCH_2016/201603%s_%s.csv.gz' % (day,ticker)), dtype='str', skip_header = 1, delimiter = ',')

# -----------------------------------------------------------------------------------------------------------------------        

        ### Lists of times, ids etc.
        times_= array([int(mytime) for mytime in data[:,0]])
        ids_ = array([int(myid) for myid in data[:,2]])
        types_ = array([1 * (mytype == 'B') + 
                        2 * (mytype == 'S') +
                        3 * (mytype == 'E') +
                        4 * (mytype == 'C') +
                        5 * (mytype == 'F') +
                        6 * (mytype == 'D') +
                        7 * (mytype == 'X') +
                        8 * (mytype == 'T') for mytype in data[:,3]])
        volumes_ = array([int(myvolume) for myvolume in data[:,4]])
        prices_ = array([int(myprice) for myprice in data[:,5]])

        ids = ids_[types_<7]
        times = times_[types_<7]
        types = types_[types_<7]
        volumes = volumes_[types_<7]
        prices = prices_[types_<7]

# -----------------------------------------------------------------------------------------------------------------------        

        ### reference lists
        prices_ref = 1 * prices
        types_ref = 0 * types
        times_ref = 0 * times
        volumes_ref = 0 * types
        index_ref= 0 * types
        newids = {}
        insertnr = {}
        hv = 0
        hv_prices = prices[types < 3]
        hv_types = types[types < 3]
        hv_times = times[types < 3]
        hv_volumes = volumes[types < 3]

# -----------------------------------------------------------------------------------------------------------------------

        for iii in range(len(ids)):
            if (types[iii] < 3):           ### Insertion
                newids[ids[iii]] = hv
                insertnr[ids[iii]] = iii
                hv += 1
            else: 
                prices_ref[iii] = hv_prices[newids[ids[iii]]]
                types_ref[iii] = hv_types[newids[ids[iii]]]
                times_ref[iii] = hv_times[newids[ids[iii]]]
                index_ref[iii] = insertnr[ids[iii]]
                volumes_ref[iii] = hv_volumes[newids[ids[iii]]]

# -----------------------------------------------------------------------------------------------------------------------

        ### minimum and maximum trade price
        minP = round(0.9 * (1. * prices_ref[types == 5] / 10000).min(),2)
        maxP = round(1.1 * (1. * prices_ref[types == 5] / 10000).max(),2)
        valuesP = minP + 0.01 * arange(int((maxP - minP) / 0.01))#arange(minP,maxP,0.01)
        maxP = valuesP.max()

# -----------------------------------------------------------------------------------------------------------------------

        ### construct quotes and spread
        nAsk = 0 * valuesP
        nAsk[-1] = 1
        nBid = 0 * valuesP
        nBid[0] = 1
        bestAsk = 10000000.
        bestAskOld = 10000000.
        bestBid = 0.
        bestBidOld = 0.
        bestAsks = []
        bestBids = []
        bestTimes = []

# -----------------------------------------------------------------------------------------------------------------------

        for iii in range(len(ids)):
            ### Incomming limit orders
            myPriceIndex = int(round(1. * (1.* prices_ref[iii] / 10000 - minP) / 0.01))
            bestAskOld = 1 * bestAsk
            bestBidOld = 1 * bestBid
            if (0 <= myPriceIndex and myPriceIndex < len(valuesP)):
                if types[iii]==2:
                    if (nAsk[myPriceIndex] == 0):
                        bestAsk = min(bestAsk,valuesP[myPriceIndex])
                    nAsk[myPriceIndex] += 1
                if types[iii]==1:
                    if (nBid[myPriceIndex] == 0):
                        bestBid = max(bestBid,valuesP[myPriceIndex])
                    nBid[myPriceIndex] += 1
                ### limit orders completely leaving
                if (types[iii] == 5 or types[iii] == 6):
                    if (types_ref[iii] == 2):
                        nAsk[myPriceIndex] -= 1
                        if (nAsk[myPriceIndex] == 0 and valuesP[myPriceIndex] == bestAsk):
                            bestAsk = valuesP[nAsk > 0].min()
                    else:
                        nBid[myPriceIndex] -= 1
                        if (nBid[myPriceIndex] == 0 and valuesP[myPriceIndex] == bestBid):
                            bestBid = valuesP[nBid > 0].max()

            if (bestAsk != bestAskOld or bestBid != bestBidOld):

                bestTimes.append(times[iii])
                bestAsks.append(bestAsk)
                bestBids.append(bestBid)
                bestAskOld = bestAsk
                bestBidOld = bestBid

# -----------------------------------------------------------------------------------------------------------------------

        spread_ = array(bestAsks) - array(bestBids)
        timesS = array(bestTimes)
        midpoint_ = 1. * (array(bestAsks) + array(bestBids)) / 2
        day_times_ind = (1. * timesS / 3600 / 1000 > 9.5) * (1. * timesS / 3600 / 1000 < 16) > 0
        
        spread = spread_[day_times_ind]
        midpoint = midpoint_[day_times_ind]
        times_spread = 1. * timesS[day_times_ind] / 3600 / 1000
        
        title('%s' %ticker, fontsize=40)
        plot(times_spread, midpoint, label=('Day %s' %day))
        xlabel('Time', fontsize=35)
        ylabel('Price', fontsize=35)
        legend(loc=0,fontsize=20)
        
    fig.savefig("midpoint_plot/%s.png" %ticker)       
    

In [53]:
pool = mp.Pool(processes=8)
pool.map(plot_stocks, tickers)
pool.close()
pool.join()

AMGN 07
AVGO 07
BIIB 07
ADI 07
AKAM 07
CELG 07
AAL 07
CHRW 07
BIIB 08
ADI 08
CHRW 08
AKAM 08
AMGN 08
CELG 08
BIIB 09
CHRW 09
AVGO 08
AAL 08
ADI 09
BIIB 10
AKAM 09
CHRW 10
AMGN 09
CHRW 11
BIIB 11
CELG 09
ADI 10
AVGO 09
AKAM 10
AMGN 10
CHTR 07
BMRN 07
CHTR 08
ADI 11
AVGO 10
AMGN 11
BMRN 08
AKAM 11
CHTR 09
AAL 09
CELG 10
BMRN 09
CHTR 10
AMZN 07
ADP 07
ALXN 07
AVGO 11
BMRN 10
ALXN 08
CHTR 11
AMZN 08
ADP 08
BMRN 11
CELG 11
ALXN 09
BBBY 07
CMCSA 07
AAL 10
CA 07
AMZN 09
ALXN 10
BBBY 08
CA 08
ADP 09
CERN 07
ALXN 11
BBBY 09
CA 09
AAL 11
ADP 10
AMAT 07
BBBY 10
CERN 08
CA 10
AMZN 10
BBBY 11
AMAT 08
ADP 11
AAPL 07
AMZN 11
CERN 09
CA 11
CMCSA 08
BIDU 07
COST 07
AMAT 09
BIDU 08
ATVI 07
ADSK 07
CERN 10
BIDU 09
COST 08
BIDU 10
ADSK 08
AMAT 10
CERN 11
BIDU 11
COST 09
CTXS 07
CHKP 07
ADSK 09
ATVI 08
COST 10
CTXS 08
CHKP 08
CMCSA 09
COST 11
CHKP 09
ADSK 10
AMAT 11
CTXS 09
CSCO 07
CHKP 10
CTXS 10
CHKP 11
ADSK 11
AAPL 08
DLTR 07
ATVI 09
EQIX 07
CTXS 11
CSCO 08
EQIX 08
FAST 07
DLTR 08
DISCA 07
EQIX 09
CMCSA